In [15]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import json
import re # Necessário para expressões regulares na lógica antiga

def obter_generos_oscar(anos):
    todos_os_generos = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for ano in anos:
        url = f"https://www.imdb.com/pt/event/ev0000003/{ano}/1/?ref_=ev_acd_tl_yr_1"
        print(f"\nColetando dados para o Oscar de {ano} - URL: {url}")

        dados_ano = {
            'Ano': ano,
            'Gênero do Melhor Filme': 'Não encontrado',
            'Gênero do Melhor Diretor': 'Não encontrado',
            'Gênero do Melhor Ator': 'Não encontrado',
            'Gênero do Melhor Atriz': 'Não encontrado',
            'Gênero do Melhor Ator Coadjuvante': 'Não encontrado',
            'Gênero do Melhor Atriz Coadjuvante': 'Não encontrado'
        }

        # Mapeamento de categorias para data-testid (layout novo) ou IDs de span (layout antigo)
        # Os IDs de span foram observados para anos como 1995
        categorias_map = {
            'Melhor Filme': {
                'testid': 'BestMotionPictureoftheYear'
            },
            'Melhor Diretor': {
                'testid': 'BestAchievementinDirecting',
                'old_id': 'oscar_best_director'
            },
            'Melhor Ator': {
                'testid': 'BestPerformancebyanActorinaLeadingRole',
                'old_id': 'oscar_best_actor_in_a_leading_role'
            },
            'Melhor Atriz': {
                'testid': 'BestPerformancebyanActressinaLeadingRole',
                'old_id': 'oscar_best_actress_in_a_leading_role'
            },
            'Melhor Ator Coadjuvante': {
                'testid': 'BestPerformancebyanActorinaSupportingRole',
                'old_id': 'oscar_best_actor_in_a_supporting_role'
            },
            'Melhor Atriz Coadjuvante': {
                'testid': 'BestPerformancebyanActressinaSupportingRole',
                'old_id': 'oscar_best_actress_in_a_supporting_role'
            }
        }
        
        encontrou_algum_dado_no_ano = False

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # --- Lógica para layout pós-2004 (ano >= 2005) ---
            if ano >= 2005:
                print(f"    Aplicando lógica para layout pós-2004 (ano >= 2005)...")
                for categoria_nome, ids in categorias_map.items():
                    data_testid_val = ids.get('testid')
                    if not data_testid_val: continue

                    print(f"      Buscando contêiner para a categoria: '{categoria_nome}' com data-testid: '{data_testid_val}'")
                    category_container = soup.find('section', {'data-testid': data_testid_val})

                    if category_container:
                        movie_link_tag = None
                        content_container = category_container.find('div', class_='ipc-metadata-list-item__content-container')
                        
                        if content_container:
                            if categoria_nome == 'Melhor Filme':
                                movie_link_tag = content_container.find('a', href=re.compile(r'/title/'))
                            else:
                                all_links_in_content = content_container.find_all('a', href=True)
                                for link in all_links_in_content:
                                    if '/title/' in link['href']:
                                        movie_link_tag = link
                                        break
                        
                        if movie_link_tag:
                            movie_url = "https://www.imdb.com" + movie_link_tag['href']
                            genero_encontrado = _extrair_generos_do_filme(movie_url, headers)
                            dados_ano[f'Gênero do {categoria_nome}'] = genero_encontrado
                            if genero_encontrado != 'Não encontrado':
                                encontrou_algum_dado_no_ano = True
                            print(f"        Gênero para '{categoria_nome}': {genero_encontrado}")
                        else:
                            print(f"        Link de filme para '{categoria_nome}' NÃO encontrado para o Oscar de {ano}.")
                    else:
                        print(f"        Contêiner da categoria para '{categoria_nome}' (data-testid='{data_testid_val}') NÃO encontrado.")
            
            # --- Lógica para layout pré-2005 (ano < 2005) ---
            else:
                print(f"    Aplicando lógica para layout antigo do Oscar (ano < 2005)...")
                for categoria_nome, ids in categorias_map.items():
                    # Para anos antigos, o "Melhor Filme" ainda pode usar o layout mais novo
                    # ou uma estrutura muito específica. Re-validando o Melhor Filme aqui
                    # com a estrutura de link ipc-title-link-wrapper
                    if categoria_nome == 'Melhor Filme':
                        # Esta parte é um pouco redundante mas garante que tentamos o Melhor Filme com o ipc-
                        film_link_element_bp = soup.find('a', class_='ipc-title-link-wrapper', href=re.compile(r'/title/'))
                        if film_link_element_bp:
                            movie_url = "https://www.imdb.com" + film_link_element_bp['href']
                            genero_encontrado = _extrair_generos_do_filme(movie_url, headers)
                            dados_ano['Gênero do Melhor Filme'] = genero_encontrado
                            if genero_encontrado != 'Não encontrado':
                                encontrou_algum_dado_no_ano = True
                            print(f"        Gênero para 'Melhor Filme': {genero_encontrado} (via ipc-link-wrapper)")
                            continue # Vai para a próxima categoria se Melhor Filme foi encontrado
                        else:
                            print(f"        Melhor Filme NÃO encontrado para o ano {ano} usando ipc-link-wrapper na lógica antiga.")


                    # Para as demais categorias (pessoa), priorizar a busca por ID no span
                    old_id_val = ids.get('old_id')
                    if not old_id_val: continue # Se não tem old_id para a categoria

                    print(f"      Buscando cabeçalho para a categoria: '{categoria_nome}' com ID: '{old_id_val}'")
                    category_header_span = soup.find('span', id=old_id_val)
                    
                    movie_link_tag = None
                    if category_header_span:
                        # Tentar encontrar o link do filme associado ao vencedor
                        # (Geralmente vem DEPOIS do link da pessoa, no mesmo nível ou num pai próximo)
                        # Busca o primeiro link de título APÓS o span do ID da categoria
                        movie_link_tag = category_header_span.find_next('a', href=re.compile(r'/title/'))
                        
                        if not movie_link_tag:
                            # Se não encontrou o link de título imediatamente, pode estar em um elemento pai
                            # do span da categoria, e o link do filme está em algum lugar dentro desse pai.
                            # Esta é uma busca mais ampla, pode pegar o primeiro link de título geral da seção.
                            parent_div_or_table = category_header_span.find_parent('div') or category_header_span.find_parent('table')
                            if parent_div_or_table:
                                movie_link_tag = parent_div_or_table.find('a', href=re.compile(r'/title/'))

                    if movie_link_tag:
                        movie_url = "https://www.imdb.com" + movie_link_tag['href']
                        genero_encontrado = _extrair_generos_do_filme(movie_url, headers)
                        dados_ano[f'Gênero do {categoria_nome}'] = genero_encontrado
                        if genero_encontrado != 'Não encontrado':
                            encontrou_algum_dado_no_ano = True
                        print(f"        Gênero para '{categoria_nome}': {genero_encontrado} (via ID no span/next)")
                    else:
                        print(f"        Link de filme para '{categoria_nome}' NÃO encontrado para o Oscar de {ano}.")
            
            # Adiciona os dados do ano somente se algo foi encontrado ou se queremos registrar o ano mesmo vazio
            if encontrou_algum_dado_no_ano or ano < 2005: # Inclui anos antigos mesmo que nada seja encontrado inicialmente para depuração
                todos_os_generos.append(dados_ano)
            else:
                print(f"  Nenhum dado de gênero encontrado para o ano {ano}. Ano não adicionado aos resultados.")

        except requests.exceptions.HTTPError as e:
            print(f"Erro HTTP ao acessar a URL do Oscar {ano}: {e}")
        except requests.exceptions.RequestException as e:
            print(f"Erro de requisição ao acessar a URL do Oscar {ano}: {e}")
        except Exception as e:
            print(f"Erro inesperado ao processar o Oscar de {ano}: {e}")
        finally:
            time.sleep(2) # Delay entre as páginas do Oscar

    df_generos = pd.DataFrame(todos_os_generos)
    return df_generos

# Função auxiliar para extrair gêneros de uma página de filme
def _extrair_generos_do_filme(film_url: str, headers: dict) -> str:
    print(f"      Acessando página do filme para gêneros: {film_url}")
    try:
        film_response = requests.get(film_url, headers=headers)
        film_response.raise_for_status()
        film_soup = BeautifulSoup(film_response.text, 'html.parser')
        genres = []

        # PRIORIDADE: Buscar gêneros do JSON-LD
        script_tag = film_soup.find('script', type='application/ld+json')
        if script_tag:
            try:
                json_data = json.loads(script_tag.string)
                if 'genre' in json_data:
                    if isinstance(json_data['genre'], list):
                        genres = json_data['genre']
                    elif isinstance(json_data['genre'], str):
                        genres = [json_data['genre']]
            except json.JSONDecodeError as e:
                print(f"      Erro ao decodificar JSON-LD da página do filme ({film_url}): {e}")
            except Exception as e:
                print(f"      Erro inesperado ao processar JSON-LD da página do filme ({film_url}): {e}")

        # Fallback para extração de HTML caso o JSON-LD não funcione ou não tenha o gênero
        if not genres:
            print(f"      JSON-LD não continha gêneros ou falhou. Tentando extrair do HTML...")
            # Lógica para layouts de filmes mais recentes (data-testid='genres')
            genres_container = film_soup.find('div', {'data-testid': 'genres'})
            if genres_container:
                genre_tags = genres_container.find_all('span', class_='ipc-chip__text')
                for tag in genre_tags:
                    genres.append(tag.get_text(strip=True))
            
            # Lógica para layouts de filmes mais antigos (see-more subtext ou itemprop="genre")
            if not genres:
                old_genres_div = film_soup.find('div', class_='see-more subtext')
                if not old_genres_div:
                    old_genres_div = film_soup.find('div', id='title-overview-widget')
                    if old_genres_div:
                        genre_items = old_genres_div.find_all('span', itemprop='genre')
                        for item in genre_items:
                            if item.text.strip():
                                genres.append(item.text.strip())
                if not genres and old_genres_div:
                    genre_links = old_genres_div.find_all('a', href=re.compile(r'/genre/'))
                    for link in genre_links:
                        genres.append(link.get_text(strip=True))
                
                # Tentativa final: Gêneros diretamente após um cabeçalho "Genres" ou "Gêneros"
                if not genres:
                    genres_header = film_soup.find(['h4', 'div', 'b', 'span'], string=re.compile(r'Gêneros|Genres', re.IGNORECASE))
                    if genres_header:
                        next_element = genres_header.find_next_sibling()
                        if next_element:
                            genre_links_alt = next_element.find_all('a', href=re.compile(r'/genre/'))
                            for link in genre_links_alt:
                                genres.append(link.get_text(strip=True))
        
        if genres:
            return ", ".join(genres)
        else:
            return "Não encontrado"
    
    except requests.exceptions.RequestException as e:
        print(f"      Erro ao acessar URL do filme para gêneros ({film_url}): {e}")
        return "Erro de Requisição"
    except Exception as e:
        print(f"      Erro inesperado ao processar gêneros do filme ({film_url}): {e}")
        return "Erro Inesperado"
    finally:
        time.sleep(1.5) # Delay entre as requisições de filme

# Exemplo de uso
# Agora vamos focar nos anos anteriores a 2005
anos_para_coletar = range(1929, 2026) 
df_generos_vencedores = obter_generos_oscar(anos_para_coletar)

# Salvar os dados
df_generos_vencedores.to_csv('oscar_generos_vencedores_pre_2005.csv', index=False)
print("\nDados de gênero salvos em 'oscar_generos_vencedores_pre_2005.csv'")
print(f"Total de anos com dados de gênero coletados: {len(df_generos_vencedores)}")

print("\n--- Primeiras 25 linhas dos dados de Gênero coletados ---")
print(df_generos_vencedores.head(25).to_markdown(index=False))


Coletando dados para o Oscar de 1929 - URL: https://www.imdb.com/pt/event/ev0000003/1929/1/?ref_=ev_acd_tl_yr_1
    Aplicando lógica para layout antigo do Oscar (ano < 2005)...
      Acessando página do filme para gêneros: https://www.imdb.com/pt/title/tt0017822/?ref_=ev_nom
        Gênero para 'Melhor Filme': Aventura, Drama, Romance (via ipc-link-wrapper)
      Buscando cabeçalho para a categoria: 'Melhor Diretor' com ID: 'oscar_best_director'
        Link de filme para 'Melhor Diretor' NÃO encontrado para o Oscar de 1929.
      Buscando cabeçalho para a categoria: 'Melhor Ator' com ID: 'oscar_best_actor_in_a_leading_role'
      Acessando página do filme para gêneros: https://www.imdb.com/pt/title/tt0019071/?ref_=ev_nom
        Gênero para 'Melhor Ator': Drama, Romance, Guerra (via ID no span/next)
      Buscando cabeçalho para a categoria: 'Melhor Atriz' com ID: 'oscar_best_actress_in_a_leading_role'
      Acessando página do filme para gêneros: https://www.imdb.com/pt/title/tt0018